# Wild Fires caused by the weather






# Part 1: Data Acquisition

At this part, we will acquire data by using the crawling method.
We will crawl the National Interagency Fire Center site.
The data is the USA's wildfires history.


WildFire site: https://data-nifc.opendata.arcgis.com/datasets/nifc::wfigs-wildland-fire-locations-full-history/explore?location=-0.000000%2C0.000000%2C1.36&showTable=true

# import modules (packages)

In [1]:
import bs4
import datetime as dt
import time
import numpy as np
import requests
import pandas as pd
from bs4 import BeautifulSoup
from collections import defaultdict
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Global 

In [2]:
url = "https://data-nifc.opendata.arcgis.com/datasets/nifc::wfigs-wildland-fire-locations-full-history/explore?showTable=true"


CsvName = "wildfiresFULLDATA.csv"

# scroll down function to get data: 

In [3]:
def scrollDown(driver, element):
    # Get scroll height.
     last_height = driver.execute_script("return arguments[0].scrollHeight", element)

    while True:

        # Scroll down to the bottom.
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", element)

        # Wait for full table loading
        time.sleep(1)



        # Calculate new scroll height and compare with last scroll height.
        new_height = driver.execute_script("return arguments[0].scrollHeight", element)

        if new_height == last_height:
            # Checking if we really scrolled to bottom
            table_info = driver.find_element(by=By.CLASS_NAME, value='feature-table-count').text.split()
            if table_info[1] == table_info[3]:
                print("found {} from {}".format(table_info[1], table_info[3]))
                break
            else:
                try:
                    driver.find_element(by=By.CLASS_NAME, value='loader')
                except:
                    # This site is extremly slow and sometimes it stuck at loading. To prevent it we scroll up the footer height
                    # and scroll down again
                    driver.execute_script("arguments[0].scrollTo(0, 0);", element)
                    time.sleep(2)

        last_height = new_height

# full html page using selenium

In [4]:
def HTMLContent(url):
    driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))
    driver.get(url)
    # Wait up to 10 seconds for the element with the class "infinite-scroll-container" to be present
    wait = WebDriverWait(driver, 10)
    element1 = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "infinite-scroll-container")))
    # Get table div which we will scroll
    table = driver.find_element(by=By.CLASS_NAME, value='infinite-scroll-container')
    # Scroll the table down
    scrollDown(driver, table)

    # returns the html content
    return driver.page_source

# get rows and cols from the table

In [5]:
def getColumns(table):
    columns = table.find_all("th")
    table_columns = []
    for th in columns:
        table_columns.append(th.get_text().strip())
    return table_columns

In [6]:
def getRows(table):
    table_rows = []
    rows = table.find("tbody").find_all("tr")
    for tr in rows:
        td = tr.find_all('td')
        row = [tr.get_text().strip() for tr in td]
        table_rows.append(row)
    return table_rows


# using beautifullsoup to crawl the data

In [7]:
def crawlWildFiresHistory(html_content):
    data = {}
    soup = BeautifulSoup(html_content, "html.parser")
    table = soup.find("table")
    columns = getColumns(table)
    rows = getRows(table)

    # In this section, we combine the columns and the rows to one dictionary for creating the data frame
    for i in range(len(rows[0])):
        curr_col = []
        for j in range(len(rows)):
            curr_col.append(rows[j][i])
        data[columns[i]] = curr_col.copy()

    df = pd.DataFrame(data)
    return df.to_csv(CsvName, index=False)

In [8]:
html_content = HTMLContent(url)
crawlWildFiresHistory(html_content)

In [14]:
df = pd.read_csv(CsvName)
print(df)

C:\Users\user\AppData\Local\Temp\ipykernel_22964\733223087.py:1: DtypeWarning: Columns (13,14,16,32,61,64,68,69,77,78,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CsvName)


                 X          Y  OBJECTID ABCDMisc ADSPermissionState  \
0      -104.457511  45.785037         1      NaN          CERTIFIED   
1      -118.180712  33.808985         2      NaN            DEFAULT   
2      -114.835414  48.073946         3      NaN            DEFAULT   
3      -117.153901  33.176394         4      NaN            DEFAULT   
4      -121.104180  38.834727         5      NaN            DEFAULT   
...            ...        ...       ...      ...                ...   
247867 -101.109410  42.762637    293521      NaN            DEFAULT   
247868 -116.181226  33.644994    293522      NaN            DEFAULT   
247869 -116.212353  33.731520    293523      NaN            DEFAULT   
247870 -117.311062  33.617268    293524      NaN            DEFAULT   
247871  -88.555561  30.448894    293526      NaN            DEFAULT   

        CalculatedAcres     ContainmentDateTime         ControlDateTime  \
0                 50.64  2020/08/06 23:13:07+00  2020/08/06 23:13:24+00 